TO-DO: aquest programa dóna un error amb les dimensions de la imatge que es passa a pytorch. Si es resol aquest error, el learner hauria de funcionar correctament.

En aquest últim apartat extra veurem un exemple de reinforcement learning amb un agent constituit per una xarxa neuronal convolucional. <br>

Cal notar que aquest exemple s'ha intentat construir, sense èxit, amb la llibreria ``fastai`` que hem estat fent servir durant la pràctica. No obstant, hem tingut problemes per crear el dataloader (com veurem, en el reinforcment learning no fem servir un conjunt d'entrenament predefinit, sinó que anem entrenant "sobre la marxa" a mesura que l'agent interactua amb l'entorn), així que hem optat per treballar directament amb Pytorch.

In [ ]:
!pip install gym[accept-rom-license]
!pip install gym[atari]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 9.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446676 sha256=dbf1b0f1f6fdf221c2225f91a2956be58f4a02f2125312f4c66dd2c8eb7ea343
  Stored in directory: /root/.cache/pip/wheels/b1/1f/f7/2da07cf4f81ea264bdaf043028749d88fe0c2227134a22cf80
Successfully built AutoROM.accept-rom-license
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.8 MB/s eta 0:00:00


In [ ]:
import numpy as np
import gym
from matplotlib import pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
env = gym.make("BreakoutDeterministic-v4").env

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.9/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.9/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [ ]:
print("Num. accions:", env.action_space.n)

Num. accions: 4


Recordem: accions són SERVIR, NO FER RES, DRETA i ESQUERRA.

Anem a crear una xarxa neuronal que actuarà com a *cervell* del nostre agent intel·ligent. Donada una observació (en el nostre cas corresponent a un estat), aquesta xarxa s'ocuparà d'estimar la utilitat de les parelles estat-acció:

In [ ]:
class AgentBrain(nn.Module):
  def __init__(self, lr_input):

    super(AgentBrain, self).__init__()



    self.conv1 = nn.Conv2d(3, 32, 3) # 208x158x32
    self.relu1 = nn.ReLU(inplace = True)
    self.conv2 = nn.Conv2d(32, 64, 3) # 206x156x64
    self.relu2 = nn.ReLU(inplace = True)
    self.pool1 = nn.MaxPool2d(2, 2) # 103x78x64

    self.conv3 = nn.Conv2d(64, 128, 3) #101x76x128
    self.relu3 = nn.ReLU(inplace = True)
    self.conv4 = nn.Conv2d(128, 256, 3) #99x74x256
    self.relu4 = nn.ReLU(inplace = True)
    self.pool2 = nn.MaxPool2d(2, 2) #49x37x256

    self.fc1 = nn.Linear(49 * 37 * 256, 64)
    self.relu5 = nn.ReLU(inplace = True)
    self.fc2 = nn.Linear(64, 4) #num. accions és 4, com hem vist abans.

    self.device = torch.device('cuda')
    self.optimizer = optim.Adam(self.parameters(), lr = lr_input)
    self.loss = nn.MSELoss()
    self.to(self.device)


  def forward(self, x):
    
    x = self.conv1(x)
    x = self.relu1(x)
    x = self.conv2(x)
    x = self.relu2(x)
    x = self.pool1(x)

    x = self.conv3(x)
    x = self.relu3(x)
    x = self.conv4(x)
    x = self.relu4(x)
    x = self.pool2(x)

    x = self.fc1(x)
    x = self.relu5(x)
    x = self.fc2(x)

    return x

Construim ara l'agent. A

In [ ]:
class Agent():
  def __init__(self, gamma, epsilon, lr, input_dims, batch_size, max_memory_size = 10000, eps_end = 0.01, eps_dec = 5e-4):

    self.gamma = gamma
    self.epsilon = epsilon
    self.eps_min = eps_end
    self.eps_dec = eps_dec

    self.lr = lr

    self.action_space = [0, 1, 2, 3]

    self.mem_size = max_memory_size
    self.mem_counter = 0


    self.batch_size = batch_size

    self.Q_eval = AgentBrain(lr)

    self.state_memory = np.zeros((self.mem_size, *input_dims), dtype = np.float32)
    self.new_state_memory = np.zeros((self.mem_size, *input_dims), dtype = np.float32)

    self.action_memory = np.zeros(self.mem_size, dtype=np.int32)
    self.reward_memory = np.zeros(self.mem_size, dtype=np.float32)
    self.terminal_memory = np.zeros(self.mem_size, dtype = np.bool_)


  def store_transition(self, state, action, reward, next_state, is_done):

    next_state = np.moveaxis(next_state, -1, 0)
    state = np.moveaxis(state, -1, 0)

    i = self.mem_counter % self.mem_size

    self.state_memory[i] = state
    self.new_state_memory[i] = next_state
    self.reward_memory[i] = reward
    self.action_memory[i] = action
    self.terminal_memory[i] = is_done

    self.mem_counter += 1


  def choose_action(self, obs):

    random_number = np.random.random()

    if random_number > self.epsilon:
      state = torch.tensor([obs]).to(self.Q_eval.device)
      action = torch.argmax(self.Q_eval.forward(state)).item()

    else:
      action = np.random.choice(self.action_space)

    return action


  def learn(self):
    
    if self.mem_counter < self.batch_size:
      return

    self.Q_eval.optimizer.zero_grad()

    max_mem = min(self.mem_counter, self.mem_size)

    batch_exemples = np.random.choice(max_mem, self.batch_size, replace = False)
    batch_index = np.arange(self.batch_size, dtype = np.int32)

    disp = self.Q_eval.device

    state_batch = torch.tensor(self.state_memory[batch_exemples]).to(disp)
    next_state_batch = torch.tensor(self.new_state_memory[batch_exemples]).to(disp)
    reward_batch = torch.tensor(self.reward_memory[batch_exemples]).to(disp)
    terminal_batch = torch.tensor(self.terminal_memory[batch_exemples]).to(disp)
    action_batch = self.action_memory[batch_exemples]

    q_eval = self.Q_eval.forward(state_batch)[batch_index, action_batch]
    q_next = self.Q_eval.forward(next_state_batch)
    q_next[terminal_batch] = 0.0

    q_target = reward_batch + self.gamma * torch.max(q_next, dim = 1)[0]

    loss = self.Q_eval.loss(q_target, q_eval).to(disp)
    loss.backward()
    self.Q_eval.optimizer.step()


/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Creem ara una instància de l'agent:

In [ ]:
agent = Agent(gamma = 0.99, epsilon = 1.0, lr = 1e-5, input_dims = (3, 210, 160), batch_size = 64)

Finalment creem el bucle principal del programa:

In [ ]:
scores = []

for i in range(10):
  score = 0
  done = False
  obs = env.reset()
  #obs = np.moveaxis(obs, -1, 0)

  while not done:
    action = agent.choose_action(obs)
    obs_next , reward, terminated, _ = env.step(action)
    #obs_next = np.moveaxis(obs_next, -1, 0)

    score += reward

    done = terminated

    agent.store_transition(obs, action, reward, obs_next, terminated)

    agent.learn()

    obs = obs_next

  scores.append(score)

  avg_score = np.mean(scores[-100:])

  print("Ep.", i, "score %.3f" % score, 'avg. score %.3f' % avg_score)



Hauríem de veure com a cada iteració l'score va aumentant.